# Embeddings Generation

**What:**  
This notebook focuses on generating vector embeddings using the ClimateBERT and the Word2Vec models. 

**Why:**  
Embeddings transform raw text into numerical representations that capture semantic meanings, enabling effective downstream tasks such as information retrieval and similarity/search. Generating accurate embeddings is crucial to improve the performance of these applications.

**How:**  
- We will first utilise the huggingface dataset and download and load the pretrained ClimateBERT and Word2Vec models.
- The ClimateBERT model should already covers sufficient climate-related vocabulary. We will verify whether Word2Vec covers these terms adequately and retrain it if necessary.
- Then, we will process the text data by applying these models to generate embeddings for each document or text chunk.  
- Finally, the generated embeddings will be stored in a new table in the PostgreSQL database for efficient retrieval and use in subsequent notebooks for information retrieval and other tasks.

This approach leverages knowledge from lecture topics including transformer-based contextual embeddings, static word vectors, and database management systems.


First, we will need to import necessary modules and functions from `functions.py`

In [3]:
# Necessary imports
import os
import regex as re
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pgai
import torch
import glob
import sys
from datasets import load_dataset, Features, Value

from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM

tqdm.pandas() #check if i should put it here

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

#word2vec model imports
import psycopg2
from gensim.models import KeyedVectors
from gensim.downloader import load
from gensim.utils import simple_preprocess
from collections import Counter
from gensim.models import Word2Vec


# Get the absolute path to the project root directory
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(notebook_dir, '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

# Load environment variables first
env_path = os.path.join(project_root, '.env')
load_dotenv(dotenv_path=env_path)
print(f"Loading .env from: {env_path}")

# Now create the database connection engine
engine = create_engine(os.getenv("DB_URL"))

#create session
Session = sessionmaker(bind=engine)
session = Session()

# Importing project functions
from scripts.retrieval.functions import load_climatebert_model, embed_and_store_all_embeddings, train_custom_word2vec_from_texts, download_climatebert_model, download_word2vec_model

Loading .env from: /Users/jessiefung/Desktop/DS205/group-6-final-project/.env


## 1. Read the data from the database

Read the data from the database so it's easier to access the data stored in the **climate_policy_radar** table.

In [6]:
# Read the table
df = pd.read_sql_query(
'''
SELECT * FROM climate_policy_radar WHERE "document_metadata.geographies" ~ 'MCO'
'''
, engine)
df.head()

,document_id,document_metadata.collection_summary,document_metadata.collection_title,document_metadata.corpus_type_name,document_metadata.corpus_import_id,document_metadata.category,document_metadata.description,document_metadata.document_title,document_metadata.family_import_id,document_metadata.family_slug,...,pipeline_metadata.parser_metadata.azure_model_id,pipeline_metadata.parser_metadata.parsing_date,text_block.text_block_id,text_block.language,text_block.type,text_block.type_confidence,text_block.coords,text_block.page_number,text_block.text,text_block.index
0,UNFCCC.document.i00000693.n0000,None,None,Intl. agreements,UNFCCC.corpus.i00000001.n0000,UNFCCC,<p>Monaco Biennial Transparency Report. BTR1</p>,Monaco Biennial Transparency Report. BTR1,UNFCCC.family.i00000692.n0000,monaco-biennial-transparency-report-btr1_3c56,...,prebuilt-document,2025-03-12T09:16:51.073287,11387,en,TableCell,1.0,"{{220.824,226.2024},{253.8936,226.2024},{253.8...",288.0,"34,4€",11387
1,UNFCCC.document.i00000693.n0000,None,None,Intl. agreements,UNFCCC.corpus.i00000001.n0000,UNFCCC,<p>Monaco Biennial Transparency Report. BTR1</p>,Monaco Biennial Transparency Report. BTR1,UNFCCC.family.i00000692.n0000,monaco-biennial-transparency-report-btr1_3c56,...,prebuilt-document,2025-03-12T09:16:51.073287,11412,en,TableCell,1.0,"{{220.824,269.4096},{253.8936,269.4096},{253.8...",288.0,0.01,11412
2,UNFCCC.document.i00000693.n0000,None,None,Intl. agreements,UNFCCC.corpus.i00000001.n0000,UNFCCC,<p>Monaco Biennial Transparency Report. BTR1</p>,Monaco Biennial Transparency Report. BTR1,UNFCCC.family.i00000692.n0000,monaco-biennial-transparency-report-btr1_3c56,...,prebuilt-document,2025-03-12T09:16:51.073287,11445,en,TableCell,1.0,"{{220.824,312.6168},{253.8936,312.6168},{253.8...",288.0,NO,11445
3,UNFCCC.document.i00000693.n0000,None,None,Intl. agreements,UNFCCC.corpus.i00000001.n0000,UNFCCC,<p>Monaco Biennial Transparency Report. BTR1</p>,Monaco Biennial Transparency Report. BTR1,UNFCCC.family.i00000692.n0000,monaco-biennial-transparency-report-btr1_3c56,...,prebuilt-document,2025-03-12T09:16:51.073287,11478,en,TableCell,1.0,"{{320.04,346.8096},{353.1096,346.8096},{353.10...",288.0,NO,11478
4,UNFCCC.document.i00000693.n0000,None,None,Intl. agreements,UNFCCC.corpus.i00000001.n0000,UNFCCC,<p>Monaco Biennial Transparency Report. BTR1</p>,Monaco Biennial Transparency Report. BTR1,UNFCCC.family.i00000692.n0000,monaco-biennial-transparency-report-btr1_3c56,...,prebuilt-document,2025-03-12T09:16:51.073287,11569,en,TableCell,1.0,"{{220.824,519.2568},{253.8936,519.2568},{253.8...",288.0,NO,11569


In [7]:
len(df)

170360

## 2. Embeddings generation

### 2.1 Download and load **ClimateBERT**

The code below will download and load the ClimateBERT model.

In [2]:
# Download and the ClimateBERT model
download_climatebert_model()

#Loading the ClimateBERT model and tokenizer
tokenizer, model = load_climatebert_model()

/Users/jessiefung/Desktop/DS205/group-6-final-project/.venv/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/jessiefung/Desktop/DS205/group-6-final-project/.venv/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ClimateBERT model and tokenizer downloaded and saved to /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f


### 2.2 Download and load **Word2Vec**

This Word2Vec model is untrained. We will check if training is necessary and use the trained model if needed.

In [ ]:
#Download and load the pretrained Word2Vec model
word2vec_model = download_word2vec_model()

🔄 Loading pretrained Word2Vec model: word2vec-google-news-300
✅ Model loaded!


Now check if the pre-trained Word2Vec model is able to cover climate-specific words in the climate policy radar. If they cannot be covered we would have to train the Word2Vec model.

In [ ]:
query = """
SELECT "text_block.text"
FROM climate_policy_radar
WHERE "text_block.text" IS NOT NULL
LIMIT 10000;
"""

df = pd.read_sql_query(query, engine)

# 3. Tokenize and gather all unique words
all_tokens = []
for text in df['text_block.text']:
    tokens = simple_preprocess(text)
    all_tokens.extend(tokens)

# 4. Compare to Word2Vec vocabulary
vocab = set(word2vec_model.key_to_index)
oov_words = [token for token in all_tokens if token not in vocab]

# 5. Count top missing words
oov_counter = Counter(oov_words)
most_common_oov = oov_counter.most_common(50)

# 6. Display
print("❌ Top OOV words not in Word2Vec:")
for word, count in most_common_oov:
    print(f"{word}: {count}")


❌ Top OOV words not in Word2Vec:
of: 3358
and: 2563
to: 1564
albania: 310
albanian: 159
dcm: 144
wem: 136
necp: 79
modelling: 72
ktoe: 69
implem: 57
meur: 43
tirana: 41
ghgs: 33
gwp: 33
adriatic: 32
montenegro: 28
oshee: 23
vlora: 23
albgaz: 23
pams: 18
neeap: 17
mva: 17
lulucf: 17
unfccc: 16
aee: 15
ionian: 15
ippu: 15
programme: 15
smes: 14
entso: 13
instat: 13
tpes: 13
wbif: 13
alkogap: 12
gwh: 12
elbasan: 12
hfc: 12
ktco: 11
labelling: 10
dumrea: 10
hpp: 9
iap: 9
mmr: 9
nzeb: 9
balkans: 9
escos: 9
ebrd: 8
kfw: 8
mte: 8


We can see there are some terms i.e. abbreviations and locations that are absent in word2vec. We'll train the model and also make sure the embeddings are in 768 dimension.

In [ ]:
#use the function to train the model so the absent words can be added

texts = df['text_block.text'].dropna().tolist()

important_terms = [
    "albania", "albanian", "unfccc", "gwp", "ghgs", "necp", "modelling", "ktoe", 
    "tirana", "vlora", "adriatic", "ionian", "montenegro", "albgaz", "oshee",
    "lulucf", "neeap", "wbif", "instat", "tpes", "gwh", "nzeb", "entso", "smes"
]

model = train_custom_word2vec_from_texts(
    texts=texts,
    force_include_words=important_terms
)


In [4]:
# DOUBLE CHECK IF THE MODEL IS LOADED CORRECTLY
# Load model if needed

model = Word2Vec.load("./local_model/custom_word2vec_768.model")


# List of words you want to check
words_to_check = [
    "albania", "unfccc", "gwp", "oshee", "tirana", "ktoe", "neeap", "smes"
]

# Check dimensionality and coverage
for word in words_to_check:
    if word in model.wv:
        vec = model.wv[word]
        print(f"✅ '{word}' in vocab | Dim: {len(vec)}")
    else:
        print(f"❌ '{word}' NOT in vocabulary")

✅ 'albania' in vocab | Dim: 768
✅ 'unfccc' in vocab | Dim: 768
✅ 'gwp' in vocab | Dim: 768
✅ 'oshee' in vocab | Dim: 768
✅ 'tirana' in vocab | Dim: 768
✅ 'ktoe' in vocab | Dim: 768
✅ 'neeap' in vocab | Dim: 768
✅ 'smes' in vocab | Dim: 768


## 3. Embedding all documents for all countries

Generate embeddings for all documents and upload them into the database.

**IMPORTANT THING TO DO BEFORE RUNNING THE CODE BELOW:**

A new table is needed, this will be created through the `create_table.sql` file. Steps to run it:

1. Go to `create_table.sql` and run the query to create the table
2. Remember to select the Postgres Server at the bottom, and then highlight the code and right click to run query


This will create a new table in the database. The file also includes a commented `DROP TABLE IF EXISTS document_embeddings;` line if the table does not appear. You can use it by uncommenting the line but try not to use it after the data are uploaded because it will drop all existing data. It should be used with cautious. After creating the table, then run the code below to generate embeddings and store them into the database.



In [ ]:
df=df[:50000]

In [13]:
#Embedding and storing all embeddings in the database

embed_and_store_all_embeddings(df, engine)


/Users/jessiefung/Desktop/DS205/group-6-final-project/scripts/retrieval/functions.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jessiefung/Desktop/DS205/group-6-final-project/scripts/retrieval/functions.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_codes = df["country_code"].dropna().unique()


Filtering by country:   0%|          | 0/1 [00:00<?, ?it/s]

Processing all countries:   0%|          | 0/1 [00:00<?, ?it/s]

Embedding MCO:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

...

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

Uploading MCO:   0%|          | 0/50000 [00:00<?, ?it/s]


✅ All ClimateBERT and Word2Vec embeddings uploaded directly.
